<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/C%C3%B3digos%20convencionales/Codigos_Hamming_(15%2C_11)_y_datos_crudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Codigos Hamming (15, 11) y datos crudos

import numpy as np
import csv
import pandas as pd
def guardar_matriz_en_csv(matriz, nombre_archivo):
    # Convertir la matriz a un DataFrame de pandas
    df = pd.DataFrame(matriz.T)  # Transponer para que cada columna corresponda a una probabilidad

    # Asignar nombres a las columnas como Probabilidad 1, Probabilidad 2, etc.
    df.columns = [f'Probabilidad {i + 1}' for i in range(matriz.shape[0])]

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(nombre_archivo, index=False)
def canal_bsc(mensaje, ruido):
    channel_message = np.bitwise_xor(mensaje, ruido)
    return channel_message
def hamming_data(bits):
    matrix_dimension = int(np.ceil(np.log2(len(bits))))
    potencias_de_dos = [2 ** i for i in range(matrix_dimension + 1)]
    posiciones_info = [i for i in range(1, 2 ** matrix_dimension + 1) if i not in potencias_de_dos]
    return posiciones_info, matrix_dimension
def calculate_ber(original_bits, decoded_bits):
    error_count = np.sum(original_bits != decoded_bits)
    ber = error_count / len(original_bits)
    return ber
def hamming_decode(Rx_bits, posiciones_info):
    uncoded_bits = []
    parity_check = 0
    for i, bit in enumerate(Rx_bits):
        if bit:
            parity_check ^= i
    if parity_check != 0:
        Rx_bits[parity_check] ^= 1
    for pos in posiciones_info:
        uncoded_bits.append(Rx_bits[pos])

    return uncoded_bits
def hamming_encode(bits, posiciones_info, matrix_dimension):
    bits_data = list(enumerate(bits))
    matriz_hamming = np.zeros(2**matrix_dimension, dtype=int)
    bit_activados = np.zeros((matrix_dimension, ((2*matrix_dimension)-1)), dtype=int)
    flags_paridad = np.zeros(matrix_dimension, dtype=int)
    bits_paridad = np.zeros(matrix_dimension, dtype=int)
    for i, pos in enumerate(posiciones_info): matriz_hamming[pos] = bits_data[i][1]
    for i, pos in enumerate(posiciones_info):
        pos_bin = format(pos, '04b')
        for j in range(matrix_dimension):
            if pos_bin[(matrix_dimension-1) - j] == '1':
                bit_activados[j, flags_paridad[j]] = matriz_hamming[pos]
                flags_paridad[j] += 1
    # for i in range(0, len(matriz_hamming), 4):print(matriz_hamming[i:i+4])
    for j in range(matrix_dimension):
        for i in bit_activados[j]:
            bits_paridad[j] ^= i
        matriz_hamming[2 ** j] = bits_paridad[j]
    return matriz_hamming
def tanda_hamming(batch_size, noise_block_len, block_len, qs, _, name):
    ber_q = np.zeros((len(qs), batch_size))
    for j, q in enumerate(qs):
        data = np.random.randint(0, 2, (batch_size, block_len, 1))
        noise = np.random.binomial(1, q, size=(batch_size, noise_block_len, 1)).astype(int)
        for i in range(0, batch_size):
            raw_data = (data[i:i + 1, :, :]).squeeze().squeeze()
            batch_noise = (noise[i:i + 1, :, :]).squeeze().squeeze()
            posiciones_info, matrix_dimension = hamming_data(raw_data)
            encoded_vector = hamming_encode(raw_data, posiciones_info, matrix_dimension)
            channel_vector = canal_bsc(encoded_vector, batch_noise)
            decoded_vector = hamming_decode(channel_vector, posiciones_info)
            ber = calculate_ber(raw_data, decoded_vector)
            ber_q[j][i] = ber
    return ber_q
def guardar_csv(data, filename):
    filepath = filename  # Ajusta la ruta según dónde quieras guardar el archivo
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Q values', 'Average BER'])
        for row in data:
            writer.writerow(row)
def tanda_raw(batch_size, block_len, qs, _, name):
    ber_q = np.zeros((len(qs), batch_size))
    for j, q in enumerate(qs):
        data = np.random.randint(0, 2, (batch_size, block_len, 1))
        noise = np.random.binomial(1, q, size=(batch_size,block_len, 1)).astype(int)
        for i in range(0, batch_size):
            raw_data = (data[i:i + 1, :, :]).squeeze().squeeze()
            batch_noise = (noise[i:i + 1, :, :]).squeeze().squeeze()
            channel_vector = canal_bsc(raw_data, batch_noise)
            ber = calculate_ber(raw_data, channel_vector)
            ber_q[j][i] = ber
    return ber_q

if __name__ == "__main__":

    block_len = 11
    noise_block_len = 16
    batch_size = 100
    code_rate = 3
    num_block = 100000
    num_test_batch = int(num_block / batch_size)

    qs = np.linspace(0.0001, 0.15, 20)
    arrays_q_raw = []
    arrays_q_HAM = []
    print(f"Total:", num_test_batch, "tandas")
    for _ in range(num_test_batch):
        ber_q_HAM = tanda_hamming(batch_size, noise_block_len, block_len, qs, _,"H1.csv")
        arrays_q_HAM.append(ber_q_HAM)
        ber_q_raw = tanda_raw(batch_size, block_len, qs, _, "RAW1.csv")
        arrays_q_raw.append(ber_q_raw)

    qs_reshaped = qs.reshape(20, 1)

    HAM_stack = np.hstack(arrays_q_HAM)
    HAM_data = np.hstack((qs_reshaped, HAM_stack))
    HAM_data_transposed = HAM_data.T
    guardar_matriz_en_csv(HAM_data, "raw50.csv")

    raw_stack = np.hstack(arrays_q_raw)
    raw_data = np.hstack((qs_reshaped, raw_stack))
    RAW_data_transposed = raw_data.T
    guardar_matriz_en_csv(raw_data, "raw50.csv")


    print(raw_data.shape)
    print(HAM_data.shape)
